In [75]:
from matplotlib import pyplot
import numpy as np
import os
import torch
import torchvision.transforms.functional as fn
import torchvision
from torchvision.io import read_image
from PIL import Image
from torchvision.transforms import ToTensor
from torch.nn.functional import one_hot as one_hot_encoding
import matplotlib.pyplot as pltd
import import_ipynb

from torchsummary import summary
import landscapedataset
import re
from torch.utils.data import Dataset, DataLoader


In [76]:
patch = 256
category = {
            'forest': 0, 'buildings': 1, 'river': 2, 'mobilehomepark': 3,
             'harbor': 4, 'golfcourse': 5, 'agricultural': 6, 
             'runway': 7, 'baseballdiamond' : 8, 'overpass' : 9, 'chaparral':10, 
             'tenniscourt':11, 'intersection':12, 'airplane':13, 'parkinglot':14, 
             'sparseresidential':15, 'mediumresidential':16, 'denseresidential':17, 
             'beach':18, 'freeway':19, 'storagetanks':20
}
path = "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset"

In [77]:
# dataset I created 
# easier to use dataloader 

class LandscapeDataset(Dataset):
    # here label is a 100 element tensor
    def __init__(self, label, img_dir, transform=None, target_transform=None):
        self.img_labels = label
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        path_to_data = self.img_dir + "/" + os.listdir(self.img_dir)[idx]
        image = Image.open(path_to_data)
        
        # make it a tensor
        image = ToTensor()(image)
        label = self.img_labels[idx]
        
        if self.transform:
            image = self.transform(image,size=[patch//4, patch//4])
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [78]:
category = {
            'forest': 0, 'buildings': 1, 'river': 2, 'mobilehomepark': 3,
             'harbor': 4, 'golfcourse': 5, 'agricultural': 6, 
             'runway': 7, 'baseballdiamond' : 8, 'overpass' : 9, 'chaparral':10, 
             'tenniscourt':11, 'intersection':12, 'airplane':13, 'parkinglot':14, 
             'sparseresidential':15, 'mediumresidential':16, 'denseresidential':17, 
             'beach':18, 'freeway':19, 'storagetanks':20
}
path = "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset"

data = os.listdir(path)
labels = torch.zeros(len(data))
# try to find the class that image belongs to
for i in range (len(data)):
    labels[i] = category[re.findall('([a-zA-Z ]+)\d*.*', data[i])[0]]
dataset = LandscapeDataset(
                            labels, 
                           "/Users/summer/desktop/CNNproj-for-landscape-images/data/UCMerced_LandUse/WholeDataset",
                            transform = fn.resize
                          )

In [79]:
training_set, validation_set, test_set = torch.utils.data.random_split(dataset, 
                                                                       [1260, 420, 420], 
                                                                       generator=torch.Generator().manual_seed(42)
                                                                      )
# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=32, shuffle=True,drop_last =True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=32, shuffle=False,drop_last =True)

In [33]:
class CNN(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, input):
        pass
        return input

In [108]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3),padding="same")
        self.conv2 = nn.Conv2d(32, 64, (3,3),padding='same')
        self.conv3 = nn.Conv2d(64,128,(3,3), padding="same")
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.fc = nn.Linear(8192, 21)
        self.dropout = nn.Dropout(p=0.5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        
        return x
    

In [136]:
summary(Net(), input_size=( 3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             896
              ReLU-2           [-1, 32, 64, 64]               0
         MaxPool2d-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          18,496
              ReLU-5           [-1, 64, 32, 32]               0
         MaxPool2d-6           [-1, 64, 16, 16]               0
            Conv2d-7          [-1, 128, 16, 16]          73,856
              ReLU-8          [-1, 128, 16, 16]               0
         MaxPool2d-9            [-1, 128, 8, 8]               0
          Flatten-10                 [-1, 8192]               0
           Linear-11                   [-1, 21]         172,053
          Dropout-12                   [-1, 21]               0
          Softmax-13                   [-1, 21]               0
Total params: 265,301
Trainable params:

In [34]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

In [110]:
model = Net()

In [111]:
running_loss = 0.
last_loss = 0.
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
# Here, we use enumerate(training_loader) instead of
# iter(training_loader) so that we can track the batch
# index and do some intra-epoch reporting
for i, data in enumerate(training_loader):
    # Every data instance is an input + label pair
    inputs, labels = data

    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = model(inputs)

    # Compute the loss and its gradients
    outputs = torch.argmax(outputs, dim=1)
    outputs = outputs.to(torch.float)
    outputs.requires_grad = True
    
    loss = loss_fn(outputs, labels)
    print(loss)

    loss.backward()

    # Adjust learning weights
    optimizer.step()

    # Gather data and report
    running_loss += loss.item()
    if i % 10  == 9:
        last_loss = running_loss / 10 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        running_loss = 0.

tensor(3720.4043, grad_fn=<DivBackward1>)
tensor(1894.0665, grad_fn=<DivBackward1>)
tensor(1989.0452, grad_fn=<DivBackward1>)
tensor(3982.6130, grad_fn=<DivBackward1>)
tensor(3937.0732, grad_fn=<DivBackward1>)
tensor(2648.7676, grad_fn=<DivBackward1>)
tensor(4000.3257, grad_fn=<DivBackward1>)
tensor(3687.7312, grad_fn=<DivBackward1>)
tensor(2623.5073, grad_fn=<DivBackward1>)
tensor(1959.9724, grad_fn=<DivBackward1>)
  batch 10 loss: 3044.3506469726562
tensor(1656.9861, grad_fn=<DivBackward1>)
tensor(2367.3284, grad_fn=<DivBackward1>)
tensor(3456.4707, grad_fn=<DivBackward1>)
tensor(3652.4890, grad_fn=<DivBackward1>)
tensor(4831.3623, grad_fn=<DivBackward1>)
tensor(3008.7566, grad_fn=<DivBackward1>)
tensor(2198.7393, grad_fn=<DivBackward1>)
tensor(1785.4066, grad_fn=<DivBackward1>)
tensor(3381.3853, grad_fn=<DivBackward1>)
tensor(3651.7817, grad_fn=<DivBackward1>)
  batch 20 loss: 2999.070593261719
tensor(3563.2051, grad_fn=<DivBackward1>)
tensor(2272.5081, grad_fn=<DivBackward1>)
tenso

In [52]:
# category_names = [
#     'agricultural',
#     'forest',
#     'beach',
#     'runway',
#     'river',
# ]
# num_images = 100
# nb_classes = len(category_names)
# patch_size = 256
# channels = 3

In [ ]:
# # some experiments 
# new_image = Image.open("/Users/summer/desktop/machine learning project/data/UCMerced_LandUse/Images/agricultural/agricultural00.tif")
# image = ToTensor()(new_image)
# print(image.shape)
# new_image = fn.resize(image, size=[patch_size//4, patch_size//4])

In [ ]:
# # first we take out the data and transform them 
# X = []
# y = []

# for i in range(len(category_names)):
#     for j in range(num_images):
#         image = Image.open("/Users/summer/desktop/machine learning project/data/UCMerced_LandUse/Images/{}/{}{:02d}.tif".format(category_names[i],category_names[i],j))
        
#         # make it a tensor
#         image = ToTensor()(image)
#         # resize it 
#         image = fn.resize(image, size=[patch_size//4, patch_size//4])

#         image_arr = np.asarray(image)

#         X.append(image_arr)
#         y.append([i])

        

# X = np.array(X)
# y = np.array(y)
# X = torch.from_numpy(X)
# y = torch.from_numpy(y)

# # we don't want labels to be 1, 2 ,3 ,
# # we want it in the format of one hot encoding
# y = one_hot_encoding(y)

# torch.save(X, "/Users/summer/desktop/machine learning project/datasets.pt")
# torch.save(y, "/Users/summer/desktop/machine learning project/labels.pt")


In [ ]:
# # now we have to split the dataset into training, validation and testing 
# dataset_length = X.shape[0]
# shuffled_indices = np.arange(dataset_length)

In [ ]:
# np.random.seed(100)
# np.random.shuffle(shuffled_indices)

In [ ]:
# training_indices, validation_indices, test_indices = shuffled_indices[0:int(dataset_length*0.6)],\
#                                                      shuffled_indices[int(dataset_length*0.6):int(dataset_length * 0.8)],\
#                                                      shuffled_indices[int(dataset_length * 0.8):]

In [ ]:
# training_set, validation_set, test_set = X[training_indices],X[validation_indices],X[test_indices]
# training_label, validation_label, test_label = y[training_indices], y[validation_indices], y[test_indices]
# print(training_set.shape)

In [83]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3),padding="same")
        self.conv2 = nn.Conv2d(32, 64, (3,3),padding='same')
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2), stride=(2,2))
        self.fc = nn.Linear(16384, 21)
        self.dropout = nn.Dropout(p=0.5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x) # flatten all dimensions except batch
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        
        return x
    

In [ ]:
loss = nn.CrossEntropyLoss()
print(loss(output, int_label))

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
output.backward()


In [61]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
print(output)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

tensor([[-3.1539, -0.5191, -2.0220,  1.0171, -1.1370],
        [ 0.1893,  2.3678,  0.0744,  1.1048,  0.0262],
        [ 0.3962, -0.3478, -1.3087, -0.7338,  0.1521]], requires_grad=True)
tensor([0, 4, 0])
tensor(2.7758, grad_fn=<NllLossBackward0>)


In [128]:
dataset[0]
dataset[1]

(tensor([[[0.8314, 0.8294, 0.8167,  ..., 0.6931, 0.6637, 0.6304],
          [0.8314, 0.8216, 0.8098,  ..., 0.6255, 0.6441, 0.7108],
          [0.8186, 0.8108, 0.8137,  ..., 0.6647, 0.7422, 0.7304],
          ...,
          [0.7990, 0.8118, 0.7931,  ..., 0.6196, 0.6255, 0.6137],
          [0.7892, 0.7912, 0.7431,  ..., 0.6147, 0.6206, 0.6363],
          [0.7902, 0.8167, 0.7637,  ..., 0.6088, 0.6098, 0.6373]],
 
         [[0.8314, 0.8304, 0.8176,  ..., 0.7069, 0.6765, 0.6412],
          [0.8314, 0.8235, 0.8137,  ..., 0.6422, 0.6618, 0.7324],
          [0.8167, 0.8147, 0.8157,  ..., 0.6873, 0.7510, 0.7461],
          ...,
          [0.8088, 0.8127, 0.7961,  ..., 0.6422, 0.6382, 0.6324],
          [0.7931, 0.7990, 0.7471,  ..., 0.6333, 0.6333, 0.6461],
          [0.7971, 0.8206, 0.7588,  ..., 0.6265, 0.6314, 0.6588]],
 
         [[0.7922, 0.7863, 0.7784,  ..., 0.6343, 0.6059, 0.5784],
          [0.7843, 0.7755, 0.7667,  ..., 0.5696, 0.6059, 0.6765],
          [0.7706, 0.7696, 0.7686,  ...,

In [143]:
torch.mean(dataset[0][0],dim=1)
print(
torch.mean(dataset[0][0],dim=1)
)
print(dataset[0][0])

tensor([[0.3320, 0.3468, 0.3189, 0.3240, 0.3231, 0.3258, 0.3291, 0.3215, 0.3272,
         0.3217, 0.3239, 0.3341, 0.3352, 0.3416, 0.3394, 0.3317, 0.3315, 0.3268,
         0.3275, 0.3264, 0.3311, 0.3228, 0.3240, 0.3242, 0.3248, 0.3210, 0.3220,
         0.3184, 0.3252, 0.3264, 0.3258, 0.3222, 0.3196, 0.3221, 0.3290, 0.3230,
         0.3260, 0.3242, 0.3202, 0.3189, 0.3180, 0.3178, 0.3207, 0.3203, 0.3204,
         0.3153, 0.3204, 0.3167, 0.3185, 0.3244, 0.3265, 0.3344, 0.3372, 0.3462,
         0.3323, 0.3307, 0.3246, 0.3138, 0.3243, 0.3276, 0.3248, 0.3324, 0.3307,
         0.3349],
        [0.3437, 0.3597, 0.3354, 0.3425, 0.3424, 0.3435, 0.3471, 0.3394, 0.3466,
         0.3449, 0.3462, 0.3564, 0.3566, 0.3593, 0.3569, 0.3488, 0.3511, 0.3497,
         0.3489, 0.3459, 0.3500, 0.3432, 0.3449, 0.3462, 0.3472, 0.3409, 0.3414,
         0.3386, 0.3461, 0.3478, 0.3492, 0.3463, 0.3436, 0.3468, 0.3531, 0.3451,
         0.3455, 0.3482, 0.3456, 0.3442, 0.3432, 0.3459, 0.3490, 0.3475, 0.3472,
         0

In [155]:
temp = torch.tensor([[[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]],[[11.,12.,13.],[14.,15.,16.],[17.,18.,19.]]])


tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.]],

        [[11., 12., 13.],
         [14., 15., 16.],
         [17., 18., 19.]]])


In [160]:
torch.mean(temp, dim = 0)

tensor([[ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]])